In [ ]:
m_demog = pd.merge(m, users, left_on='userid', right_on='user_id')
m_demog['age'] = (2020 - m_demog.birth_date + 2.5) 

In [ ]:
def plot_survey_response_per_vitals_gender(vital_key):
    f, axarr = plt.subplots(2, 3, sharex=True, figsize=(7,5))

    bins = 50
    question_keys = ['q49', 'q50', 'q54', 'q55', 'q56', 'total_wellbeing']
    marker = 0

    for question_key, ax in zip(question_keys, axarr.flatten()):
        for gender in ('M', 'F'):
            df = m_demog[m_demog.salutation == gender]
            df = df[[vital_key, question_key]].dropna()

            df['bins'] = pd.cut(df[vital_key], bins)
            df.bins = df['bins'].apply(lambda x: x.left + 0.5 * (x.right - x.left))

            df = df.groupby(['bins'])[question_key].agg(['mean', 'count', 'std'])
            df['err'] = 1.96 * df['std'] / np.sqrt(df['count'])
            df = df[df['count'] > 50]

            ax.errorbar(df.index, df['mean'], yerr=df['err'], fmt='o', markersize=5, label=gender)
            
        ax.set_title('\n'.join(wrap(wording[question_key], 30)), size=10)

        if (vital_key == 'social_jetlag') or ('difference' in vital_key):
            if marker != 'None':
                ax.axvline(marker, c='k', ls=':')

        hide_and_move_axis(ax)
        
    ax.legend()
    for ax in axarr[1]:
        ax.set_xlabel(label[vital_key])

    plt.tight_layout()
    plt.savefig(f'../output/gender_survey_response_per_{vital_key}.jpg', dpi=400)
    
    
for vital_key in label.keys():
    plot_survey_response_per_vitals_gender(vital_key)

In [ ]:
def plot_survey_response_per_vitals_age(vital_key):
    f, axarr = plt.subplots(2, 3, sharex=True, figsize=(7,5))

    bins = 50
    question_keys = ['q49', 'q50', 'q54', 'q55', 'q56', 'total_wellbeing']
    marker = 0

    for question_key, ax in zip(question_keys, axarr.flatten()):
        for xmin, xmax in ((30, 65),) :
            df = m_demog[(m_demog.age > xmin) & (m_demog.age <= xmax)]
            df = df[[vital_key, question_key]].dropna()

            df['bins'] = pd.cut(df[vital_key], bins)
            df.bins = df['bins'].apply(lambda x: x.left + 0.5 * (x.right - x.left))

            df = df.groupby(['bins'])[question_key].agg(['mean', 'count', 'std'])
            df['err'] = 1.96 * df['std'] / np.sqrt(df['count'])
            df = df[df['count'] > 50]

            ax.errorbar(df.index, df['mean'], yerr=df['err'], fmt='o', markersize=5, label=f'{xmin} - {xmax}')
            
        ax.set_title('\n'.join(wrap(wording[question_key], 30)), size=10)

        if (vital_key == 'social_jetlag') or ('difference' in vital_key):
            if marker != 'None':
                ax.axvline(marker, c='k', ls=':')

        hide_and_move_axis(ax)
        
    ax.legend()
    for ax in axarr[1]:
        ax.set_xlabel(label[vital_key])

    plt.tight_layout()
    plt.savefig(f'../output/age_survey_response_per_{vital_key}.jpg', dpi=400)
    
    
for vital_key in label.keys():
    plot_survey_response_per_vitals_age(vital_key)

In [ ]:

shapes = gpd.read_file('../data/00_external/nuts3de.json')

m_spatial = m.groupby('userid').mean().reset_index()
m_spatial = pd.merge(m_spatial, users, left_on='userid', right_on='user_id')
m_spatial.rename(columns={'NUTS3': 'NUTS'}, inplace=True)
m_spatial['federal_states'] = m_spatial.NUTS.str[:-2]

m_spatial.NUTS = m_spatial.NUTS.str[:-1]
shapes['NUTS'] = shapes.id.str[:-1]

g = m_spatial.groupby('NUTS')[['q49', 'q50', 'q54', 'q55', 'q56', 'total_wellbeing']].mean()

NUTS = shapes[['NUTS', 'geometry']].dissolve(by='NUTS')

g = pd.merge(g.reset_index(), NUTS, left_on='NUTS', right_on='NUTS')
g = gpd.GeoDataFrame(g)

f, axarr = plt.subplots(2, 3, figsize=(12, 9))

cmap = matplotlib.colormaps['Purples']
cmap = cmap.resampled(4)

for question, ax in zip(['q49', 'q50', 'q54', 'q55', 'q56', 'total_wellbeing'], axarr.flatten()):
    c = g.plot(question, cmap=cmap, legend=True, ax=ax, vmin=g[question].quantile(.1), vmax=g[question].quantile(.9))
    #g['plot'] = g[question] > g[question].quantile(.8)
    #c = g.plot('plot', cmap=cmap, legend=True, ax=ax)
    
    NUTS1.plot(color='None', ax=ax, lw=1)
    ax.set_title('\n'.join(wrap(wording[question], 30)), size=10)
    
plt.tight_layout()